In [1]:
from maf import *

/Users/sineadwilliamson/Research/NF/normalizing_flows/data.py:122: SyntaxWarning: "is" with a literal. Did you mean "=="?
  kwargs = {'num_workers': 1, 'pin_memory': True} if device.type is 'cuda' else {}


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--train', action='store_true', help='Train a flow.')
parser.add_argument('--evaluate', action='store_true', help='Evaluate a flow.')
parser.add_argument('--restore_file', type=str, help='Path to model to restore.')
parser.add_argument('--generate', action='store_true', help='Generate samples from a model.')
parser.add_argument('--data_dir', default='./data/', help='Location of datasets.')
parser.add_argument('--output_dir', default='./results/maf')
parser.add_argument('--results_file', default='results.txt', help='Filename where to store settings and test results.')
parser.add_argument('--no_cuda', action='store_true', help='Do not use cuda.')
# data
parser.add_argument('--dataset', default='toy', help='Which dataset to use.')
parser.add_argument('--flip_toy_var_order', action='store_true', help='Whether to flip the toy dataset variable order to (x2, x1).')
parser.add_argument('--seed', type=int, default=1, help='Random seed to use.')
# model
parser.add_argument('--model', default='maf', help='Which model to use: made, maf.')
# made parameters
parser.add_argument('--n_blocks', type=int, default=5, help='Number of blocks to stack in a model (MADE in MAF; Coupling+BN in RealNVP).')
parser.add_argument('--n_components', type=int, default=1, help='Number of Gaussian clusters for mixture of gaussians models.')
parser.add_argument('--hidden_size', type=int, default=100, help='Hidden layer size for MADE (and each MADE block in an MAF).')
parser.add_argument('--n_hidden', type=int, default=1, help='Number of hidden layers in each MADE.')
parser.add_argument('--activation_fn', type=str, default='relu', help='What activation function to use in the MADEs.')
parser.add_argument('--input_order', type=str, default='sequential', help='What input order to use (sequential | random).')
parser.add_argument('--conditional', default=False, action='store_true', help='Whether to use a conditional model.')
parser.add_argument('--no_batch_norm', action='store_true')
# training params
parser.add_argument('--batch_size', type=int, default=100)
parser.add_argument('--n_epochs', type=int, default=50)
parser.add_argument('--start_epoch', default=0, help='Starting epoch (for logging; to be overwritten when restoring file.')
parser.add_argument('--lr', type=float, default=1e-4, help='Learning rate.')
parser.add_argument('--log_interval', type=int, default=1000, help='How often to show loss statistics and save samples.')
parser.add_argument('--test_labels', default=False, action='store_true', help='Whether to use pre-specified Y labels for test generation.')


_StoreTrueAction(option_strings=['--test_labels'], dest='test_labels', nargs=0, const=True, default=False, type=None, choices=None, help='Whether to use pre-specified Y labels for test generation.', metavar=None)

In [3]:
args = parser.parse_args("--train --model=maf --dataset=SYNTH --n_epochs=50 --conditional --batch_size=100 --test_labels".split())
#args = parser.parse_args("--train --model=maf --dataset=MOONS --n_epochs=50 --conditional".split())



In [4]:
args.device = torch.device('cpu')
torch.manual_seed(args.seed)
if args.conditional: assert args.dataset in ['MNIST', 'CIFAR10', 'SYNTH', 'MOONS'], 'Conditional inputs only available for labeled datasets MNIST and CIFAR10.'
train_dataloader, test_dataloader = fetch_dataloaders(args.dataset, args.batch_size, args.device, args.flip_toy_var_order)
args.input_size = train_dataloader.dataset.input_size
args.input_dims = train_dataloader.dataset.input_dims
args.cond_label_size = train_dataloader.dataset.label_size if args.conditional else None

In [5]:
model = MAF(args.n_blocks, args.input_size, args.hidden_size, args.n_hidden, args.cond_label_size,
                    args.activation_fn, args.input_order, batch_norm=not args.no_batch_norm)

In [6]:
model = model.to(args.device)
optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=1e-6)

In [7]:
args.results_file = os.path.join(args.output_dir, args.results_file)

print('Loaded settings and model:')
print(pprint.pformat(args.__dict__))
print(model)
print(pprint.pformat(args.__dict__), file=open(args.results_file, 'a'))
print(model, file=open(args.results_file, 'a'))

Loaded settings and model:
{'activation_fn': 'relu',
 'batch_size': 100,
 'cond_label_size': 1,
 'conditional': True,
 'data_dir': './data/',
 'dataset': 'SYNTH',
 'device': device(type='cpu'),
 'evaluate': False,
 'flip_toy_var_order': False,
 'generate': False,
 'hidden_size': 100,
 'input_dims': 2,
 'input_order': 'sequential',
 'input_size': 2,
 'log_interval': 1000,
 'lr': 0.0001,
 'model': 'maf',
 'n_blocks': 5,
 'n_components': 1,
 'n_epochs': 50,
 'n_hidden': 1,
 'no_batch_norm': False,
 'no_cuda': False,
 'output_dir': './results/maf',
 'restore_file': None,
 'results_file': './results/maf/results.txt',
 'seed': 1,
 'start_epoch': 0,
 'test_labels': True,
 'train': True}
MAF(
  (net): FlowSequential(
    (0): MADE(
      (net_input): MaskedLinear(in_features=2, out_features=100, bias=True, cond_features=1)
      (net): Sequential(
        (0): ReLU()
        (1): MaskedLinear(in_features=100, out_features=100, bias=True)
        (2): ReLU()
        (3): MaskedLinear(in_feature

In [8]:
train_and_evaluate(model, train_dataloader, test_dataloader, optimizer, args)

epoch   0 / 50, step    0 / 100; loss 1.1447
Evaluate (epoch 0) -- logp(x) = -1.051 +/- 0.049
epoch   1 / 50, step    0 / 100; loss 1.0360
Evaluate (epoch 1) -- logp(x) = -0.851 +/- 0.046
epoch   2 / 50, step    0 / 100; loss 0.8087
Evaluate (epoch 2) -- logp(x) = -0.757 +/- 0.043
epoch   3 / 50, step    0 / 100; loss 0.7655
Evaluate (epoch 3) -- logp(x) = -0.710 +/- 0.039
epoch   4 / 50, step    0 / 100; loss 0.7389
Evaluate (epoch 4) -- logp(x) = -0.674 +/- 0.037
epoch   5 / 50, step    0 / 100; loss 0.6927
Evaluate (epoch 5) -- logp(x) = -0.626 +/- 0.043
epoch   6 / 50, step    0 / 100; loss 0.5840
Evaluate (epoch 6) -- logp(x) = -0.584 +/- 0.040
epoch   7 / 50, step    0 / 100; loss 0.5587
Evaluate (epoch 7) -- logp(x) = -0.548 +/- 0.039
epoch   8 / 50, step    0 / 100; loss 0.6135
Evaluate (epoch 8) -- logp(x) = -0.530 +/- 0.042
epoch   9 / 50, step    0 / 100; loss 0.5485
Evaluate (epoch 9) -- logp(x) = -0.489 +/- 0.044
epoch  10 / 50, step    0 / 100; loss 0.5191
Evaluate (epoch

In [9]:
import tkinter
import matplotlib
matplotlib.use('TkAgg')

In [10]:
def plot_uncertainty(model, args, input_range=[0., 1.], train_X=None, train_Y=None, plot_dim=1):
    Xvals = np.linspace(input_range[0], input_range[1], 100)
    X = torch.Tensor(Xvals)
    X = X.view(X.shape[0], -1)
    
    Ys = np.zeros((100, X.shape[0]))
    model.eval()
    for i in range(100):
        u = model.base_dist.sample((X.shape[0], args.n_components)).squeeze()
        Y, _ = model.inverse(u, X)
        Ys[i, :] = Y.detach().numpy()[:, plot_dim]
    
    Ymean = np.mean(Ys, axis=0)
    Ysd = np.std(Ys, axis=0)
    plt.plot(Xvals, Ymean, 'k')
    plt.plot(Xvals, Ymean+Ysd, 'b')
    plt.plot(Xvals, Ymean-Ysd, 'b')
    if train_X is not None:
        if train_Y is not None:
            plt.plot(train_X, train_Y[:, plot_dim], 'rx')
    plt.show()


In [11]:
import numpy as np
data = next(iter(train_dataloader))
Y = data[0].detach().numpy()
X = data[1].detach().numpy()
plot_uncertainty(model, args, train_X=X, train_Y=Y)

In [12]:
print(blah)

NameError: name 'blah' is not defined

In [ ]:

model.eval()
n_row=10
#u = model.base_dist.sample((n_row**2, args.n_components)).squeeze()
#samples, _ = model.inverse(u)

u = model.base_dist.sample((n_row**2, args.n_components)).squeeze()
from sklearn.datasets import make_swiss_roll
rolls = make_swiss_roll(n_row**2, noise=0.05)
Xdist = D.Uniform(torch.tensor([0.0]), torch.tensor([1.0]))
X = Xdist.sample(torch.Size([n_row**2]))
X = torch.Tensor(X)
X = X.view(X.shape[0], -1)

Y, _ = model.inverse(u, X)


In [ ]:
import mpl_toolkits.mplot3d.axes3d as p3
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.view_init(7, -80)
Xn = X.numpy()
Yn = Y.detach().numpy()
ax.scatter(Xn, Yn[:, 0], Yn[:, 1])
plt.show()

In [ ]:
import numpy as np
plt.plot(Xn, 2*Xn - 1. + np.sin(15*Xn), 'x')
plt.plot(Xn, Yn[:, 1], 'x')
plt.show()

In [ ]:
plt.plot(Yn[:, 0], Yn[:, 1], 'x')
plt.show()

In [ ]:
for data in train_dataloader:
    print(data[0])

In [ ]:
import numpy as np
plt.plot(data[1].detach().numpy(), data[0].detach().numpy()[:, 1], 'x')
plt.show()

In [ ]:
plt.hist(data[0].detach().numpy()[:, 1])
plt.show()

In [ ]:
Xgen, Ygen = generate(model, train_dataloader.dataset.lam, args, Xmin=0.0, Xmax=30.0)

In [ ]:
plt.plot(Xgen.numpy(), Ygen.numpy()[:, 1], 'x')
plt.show()

In [ ]:
for data in train_dataloader:
    print(data[0])


In [ ]:
data[0].numpy()[:, 1]

In [ ]:
plt.plot(data[0].numpy()[:, 0], data[0].numpy()[:, 1], 'x')
plt.show()

In [ ]:
model.eval()
n_row=10
#u = model.base_dist.sample((n_row**2, args.n_components)).squeeze()
#samples, _ = model.inverse(u)

u = model.base_dist.sample((n_row**2, args.n_components)).squeeze()
from sklearn.datasets import make_swiss_roll
rolls = make_swiss_roll(n_row**2, noise=0.05)
X = torch.Tensor(rolls[0][:, 0])
X = X.view(X.shape[0], -1)

Y, _ = model.inverse(u, X)



In [ ]:
import mpl_toolkits.mplot3d.axes3d as p3
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.view_init(7, -80)
Yn = Y.detach().numpy()
Xn = X.numpy()
ax.scatter(Xn, Yn[:, 0], Yn[:, 1])
plt.show()

In [ ]:
plt.plot(samples.detach().numpy()[:, 0], samples.detach().numpy()[:, 1], 'x')
plt.show()

In [ ]:
import mpl_toolkits.mplot3d.axes3d as p3
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.view_init(7, -80)
X = samples.detach().numpy()
ax.scatter(X[:, 0], X[:, 1], X[:, 2])
plt.show()

In [ ]:
rolls = make_swiss_roll(1000, noise=0.05)
plt.plot(rolls[0][:, 0], rolls[0][:, 1], 'x')
plt.show()